### **Extract GEDI, Sentinel 1 and Sentinel 2 data for data paper**
## Include additional variables such as:
* kurtosis and skewness with negative and positive values,
* kurtosis and skewness with only positive values,
* New climatic variables,

We dont include beam identifier because it is not a value and generate an error when we export it.

In [ ]:
!pip install geemap

In [ ]:
!earthengine authenticate

  <span style="color:red">   Import packages</span>

In [ ]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()
from IPython.display import JSON
import math
import os

<span style="color:red">  Functions </span>

In [ ]:
# filter using quality criteria
def gedi_qa(img):

    quality = img.select('quality_flag').eq(1)

    night = img.select('solar_elevation').lte(0)

    degrade = img.select('degrade_flag').eq(0)

    urban = img.select('urban_proportion').lte(5)

    water = img.select('landsat_water_persistence').eq(0)

    forest = img.select('landsat_treecover').gt(10)

    rh98_70 =  img.select(['rh98']).lte(100)

    leafon = img.select('leaf_off_flag').eq(0)

    detectmodes = img.select('num_detectedmodes').gt(0)

    surface = img.select('surface_flag').gt(0)

    sensitivitymin = img.select('sensitivity').gte(0.95)

    return img.mask(quality).updateMask(night).updateMask(degrade).updateMask(water).updateMask(urban).updateMask(forest).updateMask(slope.lt(20)).updateMask(rh98_70).updateMask(leafon).updateMask(detectmodes).updateMask(surface).updateMask(sensitivitymin)


# exclude negative rh25
#def filter_rh25(img):
#    rh25 =  img.select(['rh25'])
#    return img.updateMask(rh25.gte(0))


# calculate CR index
#def trait_calc(img):
#    tmpres = img.select(['rh98']).subtract(img.select(['rh25']))
#    return tmpres.divide(img.select(['rh98']))


# Terrain correction
# Implementation by Andreas Vollrath (ESA), inspired by Johannes Reiche (Wageningen)
def terrainCorrection(image):
    imgGeom = image.geometry()
    srtm = ee.Image('USGS/SRTMGL1_003').clip(imgGeom) # 30m srtm
    sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

    # Article ( numbers relate to chapters)
    # 2.1.1 Radar geometry
    theta_i = image.select('angle')
    phi_i = ee.Terrain.aspect(theta_i) \
    .reduceRegion(ee.Reducer.mean(), theta_i.get('system:footprint'), 1000) \
    .get('aspect')

    # 2.1.2 Terrain geometry
    alpha_s = ee.Terrain.slope(srtm).select('slope')
    phi_s = ee.Terrain.aspect(srtm).select('aspect')

    # 2.1.3 Model geometry
    # reduce to 3 angle
    phi_r = ee.Image.constant(phi_i).subtract(phi_s)

    # convert all to radians
    phi_rRad = phi_r.multiply(math.pi / 180)
    alpha_sRad = alpha_s.multiply(math.pi / 180)
    theta_iRad = theta_i.multiply(math.pi / 180)
    ninetyRad = ee.Image.constant(90).multiply(math.pi / 180)

    # slope steepness in range (eq. 2)
    alpha_r = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

    # slope steepness in azimuth (eq 3)
    alpha_az = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

    #local incidence angle (eq. 4)
    theta_lia = (alpha_az.cos().multiply((theta_iRad.subtract(alpha_r)).cos())).acos()
    theta_liaDeg = theta_lia.multiply(180 / math.pi)

    # 2.2
    # Gamma_nought_flat
    gamma0 = sigma0Pow.divide(theta_iRad.cos())
    gamma0dB = ee.Image.constant(10).multiply(gamma0.log10())
    ratio_1 = gamma0dB.select('VV').subtract(gamma0dB.select('VH'))

    # Volumetric Model
    nominator = (ninetyRad.subtract(theta_iRad).add(alpha_r)).tan()
    denominator = (ninetyRad.subtract(theta_iRad)).tan()
    volModel = (nominator.divide(denominator)).abs()

    # apply model
    gamma0_Volume = gamma0.divide(volModel)
    gamma0_VolumeDB = ee.Image.constant(10).multiply(gamma0_Volume.log10())

    # we add a layover/shadow maskto the original implmentation
    # layover, where slope > radar viewing angle
    alpha_rDeg = alpha_r.multiply(180 / math.pi)
    layover = alpha_rDeg.lt(theta_i)

    # shadow where LIA > 90
    shadow = theta_liaDeg.lt(85)

    # calculate the ratio for RGB vis
    ratio = gamma0_VolumeDB.select('VV').subtract(gamma0_VolumeDB.select('VH'))

    output = gamma0_VolumeDB.addBands(ratio).addBands(alpha_r).addBands(phi_s).addBands(theta_iRad) \
    .addBands(layover).addBands(shadow).addBands(gamma0dB).addBands(ratio_1)

    return image.addBands(output.select(['VV', 'VH'], ['VV', 'VH']),None,True)


# power to decibels
def powerToDb(img):
    return ee.Image(10).multiply(img.log10())

# dbToPower
def dbToPower(img):
    return ee.Image(10).multiply(img.log10())

# to decibels
def toDB(img):
    ImgOut = ee.Image(img).log10().multiply(10.0)
    return ImgOut

def toNatural(img):
    return ee.Image(10.0).pow(img.select('..').divide(10.0)) \
           .addBands(img.select('angle','DOY','Year'))  \
           .copyProperties(img,['system:time_start'])

# remove edges
def maskEdgeVH(img):
    mask = img.select(['VH']).unitScale(-25, 5).multiply(255).toByte().connectedComponents(ee.Kernel.rectangle(1,1), 250)
    return img.updateMask(mask.select(['VH'])) \
            .copyProperties(img,['system:time_start'])

# remove stripes
def stripeMaskVH(im):
    # VH backscatter of stripes is often less than -25dB
    mask = ee.Image(0).where(im.select(['VH']).lte(-25), 1).Not()
    return im.updateMask(mask) \
      .copyProperties(im,['system:time_start'])

def maskEdgeVV(img):
    mask = img.select(['VV']).unitScale(-25, 5).multiply(255).toByte().connectedComponents(ee.Kernel.rectangle(1,1), 250)
    return img.updateMask(mask.select(['VV'])) \
            .copyProperties(img,['system:time_start'])

# remove stripes
def stripeMaskVV(im):
    # VH backscatter of stripes is often less than -25dB
    mask = ee.Image(0).where(im.select(['VV']).lte(-25), 1).Not()
    return im.updateMask(mask) \
      .copyProperties(im,['system:time_start'])


# mask cloud and shadows (Sentinel 2 )
def maskCloudAndShadowsSR(image):

    # select cloud probability band
    cloudProb = image.select('MSK_CLDPRB')

    # get snow probability band
    snowProb = image.select('MSK_SNWPRB')

    # select threshold for cloud probability
    cloud = cloudProb.lt(5)

    # Scene Classification Map - aims at providing a pixel classification map (cloud, cloud shadows, vegetation, soils/deserts, water, snow, etc.)
    scl = image.select('SCL')

    # create mask for scl shadows
    shadow = scl.eq(3)

    # create mask for scl cirrus
    cirrus = scl.eq(10)

    # create mask for snow
    snow = snowProb.lt(5)

    # combine filters into final mask
    mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))

    # return final image
    return image.updateMask(mask)


# calculate image percentiles
def imgperc(ind):
    # mask for values > 0 for both NDVI
    res = (spindices.updateMask(spindices.select('ndvi').gt(0))
            # mask for values > 0 for spectral index
             .updateMask(spindices.select(ind).gt(0))
           # select band for index of interest
             .select(ind)
           # calculated Global-wide percentiles for a given spectral index
             .reduceRegion(reducer = ee.Reducer.percentile([1,99]),
                # area of interest
                geometry = glb,
                # resolution (should be the original one, but it would be too computationally intensive)
                scale = 1000,
                # maxPixels to be used in the reduction
                maxPixels = 1e13,
                # number of tiles for computation
                tileScale = 4))
    # get values from server side
    return res.getInfo()


def addNDVI(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename("ndvi")
    return img.addBands(ndvi).addBands(ee.Image.constant(ee.Number.parse(img.date().format("D"))).rename('DOY').float())


def removedoy_max(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020),img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021),imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant','angle','Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_Before)).updateMask(imageDOY.select('DOY').lte(DOY_After))


def removedoy_pre(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020),img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021),imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant','angle','Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_pre)).updateMask(imageDOY.select('DOY').lte(DOY_Before))

def removedoy_post(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020),img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021),imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant','angle','Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_After)).updateMask(imageDOY.select('DOY').lte(DOY_post))


def addDates(img):
    return (img.addBands(ee.Image.constant(ee.Number.parse(img.date().format("D"))).rename('DOY').float())
       .addBands(ee.Image.constant(ee.Number.parse(img.date().format("Y"))).rename('Year').float()))


def removedoygrowing(img):
    imageDOY = img.select('DOY').where(img.select('Year').eq(2020),img.select('DOY').add(365))
    imageDOY = imageDOY.select('DOY').where(img.select('Year').eq(2021),imageDOY.select('DOY').add(731))
    imageDOY = imageDOY.addBands(img.select('constant','angle','Year'))
    return imageDOY.updateMask(imageDOY.select('DOY').gte(DOY_pre)).updateMask(imageDOY.select('DOY').lte(DOY_post))


# calculate image percentiles
def imgpercs1(img):
    res = (img
           # calculated global-wide percentiles for a given spectral index
             .reduceRegion(reducer = ee.Reducer.percentile([1,99]),
                # area of interest
                geometry = glb,
                # resolution (should be the original one, but it would be too computationally intensive)
                scale = 1000,
                # maxPixels to be used in the reduction
                maxPixels = 1e13,
                # number of tiles for computation
                tileScale = 4))
    # get values from server side
    return res.getInfo()

<span style="color:red">  Load collections </span>

In [ ]:
# grids that will serve as an index
grid1km = ee.Image("users/marcogirardello/forbiores/grid1km").rename('grid1km')
grid5km = ee.Image("users/marcogirardello/forbiores/grid5km").rename('grid5km')
grid10km = ee.Image("users/marcogirardello/forbiores/grid10km").rename('grid10km')
grid20km = ee.Image("users/g_oton/grid20km").rename('grid20km')
grid30km = ee.Image("users/g_oton/grid30km").rename('grid30km')
grid40km = ee.Image("users/g_oton/grid40km").rename('grid40km')
grid50km = ee.Image("users/g_oton/grid50km").rename('grid50km')

# Export grids
#grid = ee.FeatureCollection("users/marcogirardello/forbiores/exportgrid") # Smaller grid cells (192 tiles) (Will use this for exports!!)
#grid1 = ee.FeatureCollection("users/marcogirardello/forbiores/exportgrid1") # Larger grid cells (54 tiles)
##globalgrid = ee.FeatureCollection('users/guidolavespa2511/Cell2DegSq') # Global, later we select the tiles
globalgrid = ee.FeatureCollection('users/g_oton/GridGlobal') # Extra tiles in Europe
#globalgrid = ee.FeatureCollection('users/g_oton/gridEurope_filter')

# bounding box for Global
glb = globalgrid.geometry().bounds()

# Hansen forest cover dataset
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

# GEDI data
# 2A metrics (heights)
gedi_A = (ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")
                                .filterBounds(glb))

# 2B metrics (pai, fhd)
gedi_B = (ee.ImageCollection("LARSE/GEDI/GEDI02_B_002_MONTHLY")
                                .filterBounds(glb))

# digital elevation model (dem)
dem = ee.Image('USGS/SRTMGL1_003').select('elevation')

# calculate slope from dem
slope = ee.Terrain.slope(dem)

# sentinel 1: PLEASE NOTE THAT THIS COLLECTION IS FILTERED EARLY IN THE WORFLOW USING Global AS A BOUNDING BOX!
s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(glb)

# sentinel 2: PLEASE NOTE THAT THIS COLLECTION IS FILTERED EARLY IN THE WORFLOW USING Global AS A BOUNDING BOX!
s2 = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(glb)

# alos palsar
alos = (ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR')
                  .filter(ee.Filter.date('2019-01-01', '2020-12-31')))

<span style="color:red">  GEDI data pre-processing </span>

In [ ]:
# Heights: filter for quality criteria
gedi_A_clean = (gedi_A
                     #.map(gedi_sel)
                     .map(gedi_qa)
                     .select(['rh.*']).qualityMosaic('rh98'))
                     #.map(filter_rh25)
                     #.select(['rh.*','sensitivity']).qualityMosaic('rh98'))

# Aggreate Level 2B metrics
foliage = (gedi_B.qualityMosaic('fhd_normal')
                 #.updateMask(gedi_2020_B.qualityMosaic('fhd_normal'))
                 .select(['fhd_normal','pai','l2b_quality_flag','degrade_flag','cover']))

# --------- Internal variability metrics from L2A dataset
# Including Negative + Positive values
# skewness
skew = gedi_A_clean.reduce(ee.Reducer.skew()).rename('skew_negativevalues')

# kurtosis
kurt = gedi_A_clean.reduce(ee.Reducer.kurtosis()).rename('kurt_negativevalues')

# standard deviation
sd = gedi_A_clean.reduce(ee.Reducer.stdDev()).rename('sd_negativevalues')

# mean
mu = gedi_A_clean.reduce(ee.Reducer.mean()).rename('mu_negativevalues')

# cv
cv = sd.divide(mu).rename('cv_negativevalues')

# Including only Positive values
# convert to array image
arr = (gedi_A_clean
        .toArray())

# trick to solve the problem of pixels with certain bands masked
dummyImage = ee.Image(-3278).toArray()

arr1 = (arr.arrayMask(arr.gt(0)))

l = arr1.arrayLength(0)

arr2 = arr1.where(l.eq(0),dummyImage)

# mean
mu1 =  (arr2.arrayReduce(reducer = ee.Reducer.mean(),
                     axes = [0])
        .arrayFlatten([['0']])
        .rename('mean'))

# standard deviation
sd1 = (arr2.arrayReduce(reducer = ee.Reducer.stdDev(),
                     axes = [0])
        .arrayFlatten([['0']])
        .rename('sd'))

# coefficient of variation
cv1 = sd1.divide(mu1).rename('cv')

# skewness
skew1 = (arr2.arrayReduce(reducer = ee.Reducer.skew(),
                     axes = [0])
        .arrayFlatten([['0']])).rename('skew')


# kurtosis
kurt1 = (arr2.arrayReduce(reducer = ee.Reducer.kurtosis(),
                     axes = [0])
        .arrayFlatten([['0']])).rename('kurt')

'''OLD version
# cv
arr = (gedi_A_clean
        .toArray())

mean =  (arr.arrayMask(arr.gt(0))
        .arrayReduce(reducer = ee.Reducer.mean(),
                     axes = [0])
        .arrayFlatten([['0']])
        .rename('mean'))

sd = (arr.arrayMask(arr.gt(0))
        .arrayReduce(reducer = ee.Reducer.stdDev(),
                     axes = [0])
        .arrayFlatten([['0']])
        .rename('sd'))

cv = sd.divide(mean).rename('cv')

# skewness
skew1 = (arr.arrayMask(arr.gt(0))
        .arrayReduce(reducer = ee.Reducer.skew(),
                     axes = [0])
        .arrayFlatten([['0']])).rename('skew')


# kurtosis
kurt1 = (arr.arrayMask(arr.gt(0))
        .arrayReduce(reducer = ee.Reducer.kurtosis(),
                     axes = [0])
        .arrayFlatten([['0']])).rename('kurt')


# --------- Internal variability metrics from L2B dataset

# cv pavd
#pavd = (gedi_B.qualityMosaic('fhd_normal')
#      .select('pavd.*')
#      .toArray())

#pavd_mean = (pavd.arrayMask(pavd.gt(0))
#          .arrayReduce(reducer = ee.Reducer.mean(),
#              axes = [0])
#          .arrayFlatten([['0']])
#          .rename('pavd_mean'))

#pavd_sd = (pavd.arrayMask(pavd.gt(0))
#          .arrayReduce(reducer = ee.Reducer.stdDev(),
#              axes = [0])
#          .arrayFlatten([['0']])
#          .rename('pavd_sd'))

#pavd_cv = pavd_sd.divide(pavd_mean).rename('pavd_cv')

pavd = (gedi_B.qualityMosaic('fhd_normal')
      .select('pavd.*')
      .toArray())

dummyImage = ee.Image(-3278).toArray()

pavd1 = (pavd.arrayMask(pavd.gt(0)))

l = pavd1.arrayLength(0)

pavd2 = pavd1.where(l.eq(0),dummyImage)

pavd_mean = (pavd2.arrayReduce(reducer = ee.Reducer.mean(),
              axes = [0]).arrayFlatten([['0']])
            .rename('pavd_mean'))

pavd_sd = (pavd2.arrayReduce(reducer = ee.Reducer.stdDev(),
              axes = [0]).arrayFlatten([['0']])
            .rename('pavd_sd'))

pavd_cv = pavd_sd.divide(pavd_mean).rename('pavd_cv')


ee.Image(-3278)
'''

#---------- Put everything together

# Heights: select relevant bands
#heights = (gedi_A_clean.select(['rh25','rh50','rh75','rh98','sensitivity']))
heights = (gedi_A_clean.select(['rh25','rh50','rh75','rh98']))


# Combine everything together
strmetrs = heights.addBands(foliage).addBands(skew).addBands(kurt).addBands(sd).addBands(mu).addBands(cv).addBands(skew1).addBands(kurt1).addBands(sd1).addBands(mu1).addBands(cv1)#.addBands(pavd_cv)

# Filter for quality criteria based on level 2B metrics as well as rh98 > 5
strmetrs = (strmetrs
                .updateMask(strmetrs.select('l2b_quality_flag').eq(1))
                .updateMask(strmetrs.select('degrade_flag').eq(0))
                .updateMask(strmetrs.select('rh50').gt(0))
                .updateMask(strmetrs.select('cover').gt(0.1))
                .select(['rh25','rh50','rh75','rh98','fhd_normal','pai','cover','skew_negativevalues','kurt_negativevalues','sd_negativevalues',
                         'mu_negativevalues','cv_negativevalues','skew','kurt','sd','mean','cv']))
#,'sensitivity','skew','kurt','pavd_cv'
# rememeber to select only relevant bands so to make files smaller!!!!!!!

In [ ]:
# ---- draft Gini coefficient with arrays
#arr = (gedi_A_clean.select(['rh.*']).toArray())


#arr1 = arr.arrayMask(arr.gt(0))

#n = (ee.Image(list(range(1,101+1)))
#      .toArray()
#      .arrayMask(arr.gt(0))).rename('bands')


#n1 = (arr1.arrayReduce(reducer = ee.Reducer.count(),
#                     axes = [0])
#        .arrayFlatten([['0']])).rename('n')

#sum = (arr1.arrayReduce(reducer = ee.Reducer.sum(),
#                     axes = [0])
#        .arrayFlatten([['0']])).rename('n')

#sum_xtn = img.multiply(n).reduce(ee.Reducer.sum())

# Gini coefficient
#n = ee.Image(list(range(1,101+1)))
#sum_xtn = img.multiply(n).reduce(ee.Reducer.sum())
#G = (sum_xtn.divide(img.reduce(ee.Reducer.sum())).multiply(2)).subtract(102)
#GC = G.divide(100).rename(['gini'])


<span style="color:red"> Hansen data pre-processing <br>

In [ ]:
# get tree cover for the year 2000
treecover = gfc.select(['treecover2000'])

# get tree cover loss band
loss = gfc.select(['loss'])

# only pixels with a tree cover greater than or equal to 30%
treecover30p = treecover.updateMask(treecover.gte(30))

# filter out pixels where there was loss
tcovernoloss = treecover30p.updateMask(loss.eq(0))

# self mask tree cover layer
sf_mask = treecover.updateMask(loss.eq(0)).gte(30).selfMask()

# calculated count of pixels for different forest
patch_count = sf_mask.connectedPixelCount(maxSize = 1024,eightConnected =  False)

# filter out small patches from cover dataset
tcovernoloss = tcovernoloss.updateMask(patch_count.gt(6))

<span style="color:red">  Sentinel 2 max NDVI This is needed in order to identify the dates when  NDVI reaches it  maximum values
<br> It is the baseline for identifying a window that can be defineded as growing season. Note that this only done for
the year 2020. <br> This is considered as a central year for the period of interest </span>

In [ ]:
# set start and end dates
startYear = 2020
endYear = 2021
startDate = '2020-01-01'
endDate = '2020-12-31'
start_date = ee.Date.fromYMD(startYear, 1, 1);
end_date   = ee.Date.fromYMD(endYear, 12, 31);

# filter by date
s2filt = (s2.filterDate("2020-01-01","2020-12-31")
         # filter by cloudy pixel percentage
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70))
         .map(maskCloudAndShadowsSR)
          # add NDVI
         .map(addNDVI))

# create quality mosaic using the ndvi band
ndviCollectionMax = (s2filt
                    .qualityMosaic('ndvi')
                    .set('system:time_start',start_date.millis()))

# select only relevant bands
YearMaxComposite = ndviCollectionMax.select('ndvi','DOY')

<span style="color:red">  Sentinel 1 data pre-processing </span>

In [ ]:
# Import Sentinel-1 Collection
s1filt =  (s1.filterDate("2019-10-15","2021-03-15")
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filterMetadata('resolution_meters', 'equals' , 10)
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .map(addDates))

# apply terrain correction
s1filt1 = s1filt.map(terrainCorrection)

# select band, mask edge and destripe (?)
s1VH = (s1filt1.select(['VH','angle','DOY','Year'])
              .map(maskEdgeVH)
              .map(stripeMaskVH)
              .map(toNatural))

# select band, mask edge and destripe (?)
s1VV = (s1filt1.select(['VV','angle','DOY','Year'])
              .map(maskEdgeVV)
              .map(stripeMaskVV)
              .map(toNatural))

General settings for creating Sentinel 1 composites

In [ ]:
# create shifts in the day of the YEAR
DOY_Max = YearMaxComposite.select('DOY').add(365)
DOY_Before = YearMaxComposite.select('DOY').add(365).subtract(30)
DOY_After = YearMaxComposite.select('DOY').add(365).add(30)
# substract and add 60 days. This is needed for the calculations of bimonthly composites shifted of 2 months from the window of the maximum
DOY_pre = DOY_Before.subtract(60)
DOY_post = DOY_After.add(60)


<span style="color:red">  Create Sentinel 1 backscatter growing season averages </span>

In [ ]:
# create composite for VH band
VH_mean_gs = (toDB(s1VH.map(removedoygrowing)
               .mean())
               .select('constant')
               .rename('VH_gs')
               .updateMask(tcovernoloss.gt(0)))

# create composite for VV band
VV_mean_gs = (toDB(s1VV.map(removedoygrowing)
               .mean())
               .select('constant')
               .rename('VV_gs')
               .updateMask(tcovernoloss.gt(0)))

S1comb_gs = VH_mean_gs.addBands(VV_mean_gs)

<span style="color:red">  Create Sentinel 1 backscatter growing season standard deviation </span>

In [ ]:
VH_std = (s1VH.map(removedoygrowing)
     .map(toDB)
     .reduce(ee.Reducer.stdDev())
     .select('constant_stdDev')
     .rename('VH_std')
     .updateMask(tcovernoloss.gt(0)))

VV_std = (s1VV.map(removedoygrowing)
     .map(toDB)
     .reduce(ee.Reducer.stdDev())
     .select('constant_stdDev')
     .rename('VV_std')
     .updateMask(tcovernoloss.gt(0)))

<span style="color:red">  Create Sentinel 1 backscatter bimonthly composites. The final result includes 3 composites: <br>
i) around the maximum of the growing season ii) two three months before and three months after </span>

In [ ]:
#---- VV
# composite around the maximum (+-30)
VV_around_max = (toDB(s1VV.map(removedoy_max).mean())
                    .select('constant')
                    .rename('VV_max')
                    .updateMask(tcovernoloss.gt(0)))

# composite 2 months before the maximum - 1 month
VV_pre = (toDB(s1VV.map(removedoy_pre).mean())
                      .select('constant')
                      .rename('VV_pre')
                      .updateMask(tcovernoloss.gt(0)))

# composite 2 months after the maximum + 1 month
VV_post = (toDB(s1VV.map(removedoy_post)
                    .mean())
                    .select('constant')
                    .rename('VV_post')
                    .updateMask(tcovernoloss.gt(0)))

# create a single multiband image (for VV band)
VV_bimon = VV_around_max.addBands(VV_pre).addBands(VV_post)

#---- VH
# composite around the maximum (+-30)
VH_around_max = (toDB(s1VH.map(removedoy_max)
                        .mean())
                        .select('constant')
                        .rename('VH_max')
                        .updateMask(tcovernoloss.gt(0)))

# composite 2 months before the maximum - 1 month
VH_pre = (toDB(s1VH.map(removedoy_pre)
                  .mean())
                  .select('constant')
                  .rename('VH_pre')
                  .updateMask(tcovernoloss.gt(0)))

# composite 2 months after the maximum + 1 month
VH_post = (toDB(s1VH.map(removedoy_post)
                   .mean())
                   .select('constant')
                   .rename('VH_post')
                   .updateMask(tcovernoloss.gt(0)))

# create a single multiband image (for VH band)
VH_bimon = VH_around_max.addBands(VH_pre).addBands(VH_post)

<span style="color:red">  Average of Sentinel1 backscatter values around a 3 x 3 neighbourhood </span>

In [ ]:
S1gs_mean = S1comb_gs.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

VV_bimon_mean = VV_bimon.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

VH_bimon_mean = VH_bimon.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

S1_VV_std_mean = VV_std.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

S1_VH_std_mean = VH_std.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

<span style="color:red">  Sentinel 1 texture metrics (angular second moment, entropy and dissimilarity)</span>

In [ ]:
# reduction
#VV_perc = imgpercs1(S1_VV_Seas)
#VH_perc = imgpercs1(S1_VH_Seas)

In [ ]:
# get absolute values
#VV_percabs= list(map(abs,list(VV_perc.values())))
#VH_percabs= list(map(abs,list(VH_perc.values())))


In [ ]:
# New values 20230201
# # Europe
# # VH_percabs
# [9.129975410581038,18.86852351303534]
# # VV_percabs
# [2.1293859254278487,11.866973110221632]

# # Africa
# # VH_percabs
# [11.064158160595202,17.561419938039123]
# # VV_percabs
# [3.631432380473086,11.117899989957806]

# # South America
# # VH_percabs
# [9.688176644777993,18.561392003459073]
# # VV_percabs
# [1.8787949128205337,11.870208617451963]

# # North America
# # VH_percabs
# [9.313323542812679,20.437103309375498]
# # VV_percabs
# [-1.8722225172167073,13.121486524282194]

# # Asia (North)
# # VH_percabs
# [10.379627030106795,20.619963058225885]
# # VV_percabs
# [1.3794006142835262,13.37015515167544]

# # Asia (South East)
# #VH_percabs
# [7.629375177878414,18.37266444214037]
# #VV_percabs
# [0.3782812221545349,11.622306108239458]

# # Australia
# #VH_percabs
# [11.18714818664144,20.55816694218385]
# #VV_percabs
# [4.132728762162206,13.617688330920652]

# # Global
# # VH_percabs
VH_percabs = [7.629375177878414,20.619963058225885]
# # VV_percabs
VV_percabs = [0.3782812221545349,13.617688330920652]

In [ ]:
# calculate texture metrics using min and max as inputs!!
glcmVHG = (VH_mean_gs
            .abs()
            .unitScale(min(VV_percabs),max(VV_percabs))
            .multiply(255)
            .toByte()
            .glcmTexture(size = 3)
            .select(['VH_gs_ent','VH_gs_diss','VH_gs_asm']))

# calculate texture metrics using min and max as inputs!!
glcmVVG = (VV_mean_gs
            .abs()
            .unitScale(min(VH_percabs),max(VH_percabs))
            .multiply(255)
            .toByte()
            .glcmTexture(size = 3)
            .select(['VV_gs_ent','VV_gs_diss','VV_gs_asm']))

<span style="color:red">  Sentinel 2: data pre-processing and spectral indices </span>

In [ ]:
# filter by date
s2filt = (s2.filterDate("2020-01-01","2020-12-31")
         # filter by cloudy pixel percentage
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70))
         .map(maskCloudAndShadowsSR)
          # add NDVI
         .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B4']).rename('ndvi')))
          # add NDWI
         .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B11']).rename('ndwi')))
          # add GNDVI
         .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B3']).rename('gndvi')))
          # add  modified soil-adjusted index
          .map(lambda image:  image.addBands(image.expression('(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2',
                                                       {'NIR': image
                                                        .select('B8')
                                                        .divide(10000),'RED': image.select('B4')
                                                        .divide(10000)})
                                                        .rename('msavi')))
          # // Normalized Difference Red Edge Index
          .map(lambda image: image.addBands(image.normalizedDifference(['B8', 'B5']).rename('ndre'))))

# sdNDVI
sdNDVI = (s2filt.select('ndvi').reduce(ee.Reducer.stdDev())
          .rename('ndvi_std')
          .updateMask(tcovernoloss.gt(0)))

# combine spectral indices together
spindices = (s2filt.qualityMosaic('ndvi')
            .select('ndvi','gndvi','msavi','ndre') # quality mosaic based on ndvi and selection of indices based on max ndvi
            .addBands(s2filt.qualityMosaic('ndwi').select('ndwi'))  # quality mosaic based on ndwi and selection of indices based on max ndwi
            .updateMask(tcovernoloss.gt(0))) # mask only for forested areas which have been stable

<span style="color:red">  Sentinel 2 average value for each spectral index around a 3 x 3 neighbourhood </span>

In [ ]:
sdNDVI_mean = sdNDVI.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

spindices_mean = spindices.reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3))

<span style="color:red">  Sentinel 2 texture metrics (angular second moment, entropy and dissimilarity) </span>

In [ ]:
# Get 1st and 99th percentile for each spectral index. These values will be used for excluding possible outliers.
# Note that this step is not integrated in the workflow. The numbers are "downloaded" on the client side and will
# be supplied "manually" in subsequent steps to try and avoid an eccessive number of computations
# please note that this takes several minutes!!!

#gclperc = imgperc(ind = 'gcl')
#msaviperc = imgperc(ind = 'msavi')
#nrdeperc = imgperc(ind = 'nrde')
#ndwiperc = imgperc(ind = 'ndwi')

In [ ]:
# Europe
#gclperc = {'gclperc_p1':5.077386984092761,'gclperc_p99':9.067489994472286}
#msaviperc = {'msavi_p1':0.1972778761648679,'msavi_p99':0.7675430541335931}
#nrdeperc = {'nrde_p1':0.3691599378587612, 'nrde_p99':0.7283986178051558}
#ndwiperc = {'ndwi_p1':0.05667912692527294, 'ndwi_p99':0.5644235159673532}

############### New 2023-02-02
# Europe
#gndviperc = {'gndvi_p1':0.44175608964334715,'gndvi_p99':0.9278461046614124}
#msaviperc = {'msavi_p1':0.06712736099150668,'msavi_p99':0.75827983088609235}
#nrdeperc = {'nrde_p1':0.28822722407391234, 'nrde_p99':0.73523092777565325}
#ndwiperc = {'ndwi_p1':0.07359526312558348, 'ndwi_p99':0.860624547429399}

# Africa 1
#gndviperc = {'gndvi_p1':0.5313668051165791,'gndvi_p99':0.7987967336137987}
#msaviperc = {'msavi_p1':0.2464177318109606,'msavi_p99':0.649440590999581}
#nrdeperc = {'nrde_p1':0.36797383452330955, 'nrde_p99':0.6621098911155996}
#ndwiperc = {'ndwi_p1':0.15043560738581148, 'ndwi_p99':0.79018395627344535}

# Africa 2
#gndviperc = {'gndvi_p1':0.48017828329131335,'gndvi_p99':0.7991233177359246}
#msaviperc = {'msavi_p1':0.18881583572453371,'msavi_p99':0.6238702017278255}
#nrdeperc = {'nrde_p1':0.2881859356175888, 'nrde_p99':0.6582007033310086}
#ndwiperc = {'ndwi_p1':0.035182201759208357, 'ndwi_p99':0.790144785486991}

# South America 1
#gndviperc = {'gndvi_p1':0.467279373153995,'gndvi_p99':0.81120915074058075}
#msaviperc = {'msavi_p1':0.17603307153584956,'msavi_p99':0.6750567943895766}
#nrdeperc = {'nrde_p1':0.28810318528549587, 'nrde_p99':0.6836120707444648}
#ndwiperc = {'ndwi_p1':0.041589667500764773, 'ndwi_p99':0.7965360537924989}

# South America 2
#gndviperc = {'gndvi_p1':0.4159668308020187,'gndvi_p99':0.889193909512237}
#msaviperc = {'msavi_p1':0.08636647860240947,'msavi_p99':0.7903301322703388}
#nrdeperc = {'nrde_p1':0.24971391144109557, 'nrde_p99':0.7611736013493267}
#ndwiperc = {'ndwi_p1':0.0160525709097331, 'ndwi_p99':0.8543747507122507}

# North America 1
#gndviperc = {'gndvi_p1':0.3521237080854037,'gndvi_p99':0.9146577012531528}
#msaviperc = {'msavi_p1':0.08340089285714286,'msavi_p99':0.6972665976892826}
#nrdeperc = {'nrde_p1':0.1730374848362313, 'nrde_p99':0.7483584246258011}
#ndwiperc = {'ndwi_p1':-0.05727468973465445, 'ndwi_p99':0.876284323818437}

# North America 2
#gndviperc = {'gndvi_p1':0.2880255294633771,'gndvi_p99':0.9021463602140278}
#msaviperc = {'msavi_p1':0.032029462938718547,'msavi_p99':0.6589451012415505}
#nrdeperc = {'nrde_p1':0.2241878927791067, 'nrde_p99':0.7098238707570367}
#ndwiperc = {'ndwi_p1':0.01934969489735678, 'ndwi_p99':0.9925872331302353}

# North America 3
#gndviperc = {'gndvi_p1':0.3904445977011494,'gndvi_p99':0.8500000222159411}
#msaviperc = {'msavi_p1':0.1087753891044219,'msavi_p99':0.7995755730412649}
#nrdeperc = {'nrde_p1':0.27535529750142327, 'nrde_p99':0.7611505633982507}
#ndwiperc = {'ndwi_p1':0.08340880477094427, 'ndwi_p99':0.8127686922622125}

# Asia (North) 1
#gndviperc = {'gndvi_p1':0.4417121153489752,'gndvi_p99':0.8369995115527267}
#msaviperc = {'msavi_p1':0.16317291539380037,'msavi_p99':0.7839014547160958}
#nrdeperc = {'nrde_p1':0.30094680107712884, 'nrde_p99':0.70963251040118985}
#ndwiperc = {'ndwi_p1':0.06079134776791577, 'ndwi_p99':0.96958336997837}

# Asia (North) 2
#gndviperc = {'gndvi_p1':0.46734882968552765,'gndvi_p99':0.9151243676271155}
#msaviperc = {'msavi_p1':0.15043093642988215,'msavi_p99':0.7070805959649758}
#nrdeperc = {'nrde_p1':0.30100543582318314, 'nrde_p99':0.6969001647020747}
#ndwiperc = {'ndwi_p1':0.028747382927142354, 'ndwi_p99':0.9824219611171011}

# # Asia (South East) 1
#gndviperc = {'gndvi_p1':0.13432373711731932,'gndvi_p99':0.8115292837186249}
#msaviperc = {'msavi_p1':0.06074658651781881,'msavi_p99':0.7006597448931935}
#nrdeperc = {'nrde_p1':0.1631455200823893, 'nrde_p99':0.6813343433933581}
#ndwiperc = {'ndwi_p1':0.11837810463916164, 'ndwi_p99':0.78374486173020005}

# # Asia (South East) 2
#gndviperc = {'gndvi_p1':0.31387454932954442,'gndvi_p99':0.8633247517229327}
#msaviperc = {'msavi_p1':0.10558738446136297,'msavi_p99':0.7135026377952756}
#nrdeperc = {'nrde_p1':0.19851866372170073, 'nrde_p99':0.6969333516668358}
#ndwiperc = {'ndwi_p1':0.02237809796879827, 'ndwi_p99':0.8543002590673575}

# # Asia (South East) 3
#gndviperc = {'gndvi_p1':0.40333752400391686,'gndvi_p99':0.8503452820264187}
#msaviperc = {'msavi_p1':0.16317446769144606,'msavi_p99':0.7838963860781346}
#nrdeperc = {'nrde_p1':0.30084872674471726, 'nrde_p99':0.7096402340802026}
#ndwiperc = {'ndwi_p1':0.09924807682262672, 'ndwi_p99':0.7837626031907264}

# Australia
#gndviperc = {'gndvi_p1':0.28813301435406697,'gndvi_p99':0.940470206022187}
#msaviperc = {'msavi_p1':0.0544825354609929,'msavi_p99':0.7070796875}
#nrdeperc = {'nrde_p1':0.1986664314516129, 'nrde_p99':0.73534165399817575}
#ndwiperc = {'ndwi_p1':-0.0862469109468851, 'ndwi_p99':0.7582138758476786}

# Global
gndviperc = {'gndvi_p1':0.13432373711731932,'gndvi_p99':0.940470206022187}
msaviperc = {'msavi_p1':0.032029462938718547,'msavi_p99':0.7995755730412649}
ndreperc = {'ndre_p1':0.1631455200823893, 'ndre_p99':0.7611736013493267}
ndwiperc = {'ndwi_p1':0.0160525709097331, 'ndwi_p99':0.9925872331302353}

#Europe
#ndviperc = {'ndvi_p1':6560.210349312173/10000, 'ndvi_p99':9823.164823086387/10000}
#gclperc = {'gcl_p1':89599.64218723477/10000,'gcl_p99':7992782.15/10000}
#msaviperc = {'msavi_p1':2528.9756097560976/10000,'msavi_p99':7712.827586206897/10000}
#ndreperc = {'ndre_p1':5213.712121212121/10000, 'ndre_p99':7709.7793103448275/10000}
#ndwiperc = {'ndwi_p1':3727.4/10000, 'ndwi_p99':6703.438596491228/10000}

##### A note on the texture metrics: the data need to be rescaled between 0 and 1. We only retained pixels where both the value of the NDVI and a given spectral index are > 0 The rationale behind this choice is that any pixel with values < 0 is likely to contain things different from vegetation.

In [ ]:
# for the NDVI it is not necessary to use the percentiles
ndvi_text = (spindices.select('ndvi')
                     .updateMask(spindices.select('ndvi').gt(0))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('ndvi_asm','ndvi_ent','ndvi_diss'))

# green chlorophyll index
gndvi_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('gndvi')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('gndvi').gt(gndviperc.get('gndvi_p1')))
                     .updateMask(spindices.select('gndvi').lt(gndviperc.get('gndvi_p99')))
                     # normalise between 0 and 1
                     .unitScale(gndviperc.get('gndvi_p1'),gndviperc.get('gndvi_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('gndvi_asm','gndvi_ent','gndvi_diss'))

# modified soil-adjusted index
msavi_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('msavi')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('msavi').gt(msaviperc.get('msavi_p1')))
                     .updateMask(spindices.select('msavi').lt(msaviperc.get('msavi_p99')))
                     # normalise between 0 and 1
                     .unitScale(msaviperc.get('msavi_p1'),msaviperc.get('msavi_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('msavi_asm','msavi_ent','msavi_diss'))

# red-edge chlorophyll index
ndre_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('ndre')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('ndre').gt(ndreperc.get('ndre_p1')))
                     .updateMask(spindices.select('ndre').lt(ndreperc.get('ndre_p99')))
                     # normalise between 0 and 1
                     .unitScale(ndreperc.get('ndre_p1'),ndreperc.get('ndre_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('ndre_asm','ndre_ent','ndre_diss'))

# normalised difference water index
ndwi_text = (spindices.updateMask(spindices.select('ndvi').gt(0))
                     .select('ndwi')
                     # mask out values that are below the 1st and above the 99th percentiles
                     .updateMask(spindices.select('ndwi').gt(ndwiperc.get('ndwi_p1')))
                     .updateMask(spindices.select('ndwi').lt(ndwiperc.get('ndwi_p99')))
                     # normalise between 0 and 1
                     .unitScale(ndwiperc.get('ndwi_p1'),ndwiperc.get('ndwi_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('ndwi_asm','ndwi_ent','ndwi_diss'))


<span style="color:red">  Alos palsar data

In [ ]:
DNhh = alos.select('HH').mean().pow(2).log10().multiply(10).subtract(83)#.first() #// γ₀ = 10log₁₀(DN²) - 83.0 dB
DNhv = alos.select('HV').mean().pow(2).log10().multiply(10).subtract(83)#.mean() #// γ₀ = 10log₁₀(DN²) - 83.0 dB

In [ ]:
alos_mean = (DNhh.rename('alos_HH')
    .addBands(DNhv.rename('alos_HV'))
    .updateMask(tcovernoloss.gt(0))
    .reduceNeighborhood(reducer = ee.Reducer.mean(),
                             kernel = ee.Kernel.square(3)))

<span style="color:red">  Alos palsar data texture metrics

In [ ]:
#World
#hhperc = {‘hh_p1‘:4.386079497655538, ‘hh_p99’:12.869593224763989}
#hvperc = {‘hv_p1‘:9.814514027559, ‘hv_p99’:20.311537873137134}

#Europe
#hhperc = {'hh_p1':3.3819270635392384, 'hh_p99':12.869791473237525}
#hvperc = {'hv_p1':8.939966889141447, 'hv_p99':20.436821996775794}

#Global
hhperc = {'hh_p1':4.634242727191889, 'hh_p99':18.870941152756455}
hvperc = {'hv_p1':10.135550169048622, 'hv_p99':29.87896390839444}

alos_hh_text = (DNhh.updateMask(tcovernoloss.gt(0))
                     .abs()
                     # normalise between 0 and 1
                     .unitScale(hhperc.get('hh_p1'),hhperc.get('hh_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('HH_asm','HH_ent','HH_diss'))


alos_hv_text = (DNhv.updateMask(tcovernoloss.gt(0))
                     .abs()
                     # normalise between 0 and 1
                     .unitScale(hhperc.get('hh_p1'),hhperc.get('hh_p99'))
                     # convert to byte (needed for texture calculation)
                     .multiply(255)
                     .toByte()
                      # calculate texture metrics
                     .glcmTexture(size = 3)
                     .select('HV_asm','HV_ent','HV_diss'))

<span style="color:red">  Coherence data (source: https://www.nature.com/articles/s41597-022-01189-6) </span>

In [ ]:
# 6-day coherence data for the summer
summer_VV06_data = (ee.Image("users/marcogirardello/forbiores/coherence/summer_vv_COH06_100")
                            .updateMask(tcovernoloss.gt(0))).rename(['coh_6day_summer'])

# 12-day coherence data for the summer
summer_VV12_data = (ee.Image("users/marcogirardello/forbiores/coherence/summer_vv_COH12_100")
                            .updateMask(tcovernoloss.gt(0))).rename(['coh_12day_summer'])

<span style="color:red">  Biomass (source: https://doi.pangaea.de/10.1594/PANGAEA.894711) </span>

In [ ]:
# Climatic variables
elevation = ee.Image("USGS/SRTMGL1_003").rename('elevation')
aspect = ee.Terrain.aspect(dem)
temperature = ee.Image("WORLDCLIM/V1/BIO").select('bio01').rename('temperature')
precipitation = ee.Image("WORLDCLIM/V1/BIO").select('bio12').rename('precipitation')
soil = ee.Image("OpenLandMap/SOL/SOL_GRTGROUP_USDA-SOILTAX_C/v01").rename('soil')

ET = ee.Image("users/g_oton/ET_Zomer_v3").rename('ET')
ETsd = ee.Image("users/g_oton/ETsd_Zomer_v3").rename('ETsd')
aridity = ee.Image("users/g_oton/AridityIndex_Zomer_v3").rename('aridity')

fcover = ee.Image("users/g_oton/fcover_20200701_RT6_v111").rename('fcover')
fcover = fcover.multiply(0.004)
TCD = ee.Image(ee.ImageCollection("users/g_oton/TreeCoverDensity").max()).rename('TCD')

Biomass1 = ee.Image("users/g_oton/BiomassCCI_2018_v3_1of6").updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32().rename('biomass')
Biomass2 = ee.Image("users/g_oton/BiomassCCI_2018_v3_2of6").updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32().rename('biomass')
Biomass3 = ee.Image("users/g_oton/BiomassCCI_2018_v3_3of6").updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32().rename('biomass')
Biomass4 = ee.Image("users/g_oton/BiomassCCI_2018_v3_4of6").updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32().rename('biomass')
Biomass5 = ee.Image("users/g_oton/BiomassCCI_2018_v3_5of6").updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32().rename('biomass')
Biomass6 = ee.Image("users/g_oton/BiomassCCI_2018_v3_6of6").updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32().rename('biomass')
biomass = ee.Image(ee.ImageCollection.fromImages([Biomass1,Biomass2,Biomass3,Biomass4,Biomass5,Biomass6]).max()).rename('biomass')

Conversion Float to Integer (-2,147,483,648 to 2,147,483,648)

In [ ]:
############## Warning - mask GEDI shots - we reduce the size of the data
tcovernoloss = tcovernoloss.updateMask(strmetrs.select('rh50').gt(0))
####gedi_mask = strmetrs.select('rh50').updateMask(strmetrs.select('rh50').gt(0))

strmetrs = strmetrs.multiply(10000)
S1gs_mean = S1gs_mean.multiply(10000)
S1_VV_std_mean = S1_VV_std_mean.multiply(10000)
S1_VH_std_mean = S1_VH_std_mean.multiply(10000)
VV_bimon_mean = VV_bimon_mean.multiply(10000)
VH_bimon_mean = VH_bimon_mean.multiply(10000)
glcmVHG = glcmVHG.multiply(10000)
glcmVVG = glcmVVG.multiply(10000)
spindices_mean = spindices_mean.multiply(10000)
sdNDVI_mean = sdNDVI_mean.multiply(10000)
ndvi_text = ndvi_text.multiply(10000)
gndvitext = gndvi_text.multiply(10000)
msavi_text = msavi_text.multiply(10000)
ndre_text = ndre_text.multiply(10000)
ndwi_text = ndwi_text.multiply(10000)
alos_mean = alos_mean.multiply(10000)
alos_hh_text = alos_hh_text.multiply(10000)
alos_hv_text = alos_hv_text.multiply(10000)
aspect = aspect.multiply(10000)
slope = slope.multiply(10000)
fcover = fcover.multiply(10000)
TCD = TCD.multiply(10000)

<span style="color:red">  Put everything together </span>

In [ ]:
trainingdat = (strmetrs.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()
               .addBands(S1gs_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 mean backscatter data for growing season
               .addBands(S1_VV_std_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 temporal variability backscatter data for growing season VV polarisation
               .addBands(S1_VH_std_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 temporal variability backscatter data for growing season VH polarisation
               .addBands(VV_bimon_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 bimonthly composites VV polarisation
               .addBands(VH_bimon_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 bimonthly composites VH polarisation
               .addBands(glcmVHG.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 texture metrics VH polarisation
               .addBands(glcmVVG.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S1 texture metrics VV polarisation
               .addBands(spindices_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S2 mean values of spectral indices
               .addBands(sdNDVI_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # S2 mean values of spectral indices
               .addBands(ndvi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # ndvi texture metrics
               .addBands(gndvi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # gndvi texture metrics
               .addBands(msavi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # msavi texture metrics
               .addBands(ndre_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # rcl texture metrics
               .addBands(ndwi_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # ndwi texture metrics
               .addBands(summer_VV06_data.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # coherence summer data 6 days
               .addBands(summer_VV12_data.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # coherence summer data 12 days
               .addBands(alos_mean.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # alos palsar mean
               .addBands(alos_hh_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # alos palsar texture HH
               .addBands(alos_hv_text.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # alos palsar texture HV
               .addBands(elevation.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # elevation
               .addBands(slope.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # slope
               .addBands(aspect.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # aspect
               .addBands(biomass.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # biomass Santoro (AGB 2018 v3)
               .addBands(fcover.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # fcover Copernicus
               .addBands(TCD.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # Tree cover density Copernicus
               .addBands(soil.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # soil
               .addBands(temperature.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # temperature
               .addBands(precipitation.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # precipitation
               .addBands(ET.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # The Potential Evapo-Transpiration (ET0) Database v3
               .addBands(ETsd.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # The Potential Evapo-Transpiration (ET0) std Database v3
               .addBands(aridity.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # Global Aridity Index (Global-AI) Database v3
               .addBands(grid1km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 1 km grid
               .addBands(grid5km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 5 km grid
               .addBands(grid10km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 10 km grid
               .addBands(grid20km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 20 km grid
               .addBands(grid30km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 30 km grid
               .addBands(grid40km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32()) # 40 km grid
               .addBands(grid50km.updateMask(tcovernoloss.gt(0)).unmask(-2147483648).toInt32())) # 50 km grid



In [ ]:
#roi = ee.Geometry.Polygon([[[9.010587, 48.828667],
#  [9.072313, 48.828667],
#  [9.072313, 48.854909],
#  [9.010587, 48.854909],
#  [9.010587, 48.828667]]])

In [ ]:
#predictiondat = (S1gs_mean.toFloat() # S1 mean backscatter data for growing season
#                   .addBands(S1_VV_std_mean.toFloat()) # S1 temporal variability backscatter data for growing season VV polarisation
#                   .addBands(S1_VH_std_mean.toFloat()) # S1 temporal variability backscatter data for growing season VH polarisation
#                   .addBands(VV_bimon_mean.toFloat()) # S1 bimonthly composites VV polarisation
#                   .addBands(VH_bimon_mean.toFloat()) # S1 bimonthly composites VH polarisation
#                   .addBands(glcmVHG.toFloat()) # S1 texture metrics VH polarisation
#                   .addBands(glcmVVG.toFloat()) # S1 texture metrics VV polarisation
#                   .addBands(spindices_mean.toFloat()) # S2 mean values of spectral indices
#                   .addBands(ndvi_text.toFloat()) # ndvi texture metrics
#                   .addBands(gndvi_text.toFloat()) # gndvi texture metrics
#                   .addBands(msavi_text.toFloat()) # msavi texture metrics
#                   .addBands(rcl_text.toFloat()) # rcl texture metrics
#                   .addBands(ndwi_text.toFloat()) # ndwi texture metrics
#                   .addBands(summer_VV06_data.toFloat()) # coherence summer data 6 days
#                   .addBands(summer_VV12_data.toFloat())) # coherence summer data 12 days

<span style="color:red">  Export training and prediction data to cloud storage (only for multiple accounts!!!!!) </span>

In [ ]:
# create fake environment variables (example!)
#%env minpoly = 117
#%env maxpoly = 117
# get environment variables supplied to docker run etc. command


In [ ]:
#minpoly = int(os.environ["minpoly"])
#maxpoly = int(os.environ["maxpoly"])
#polyl = list(range(minpoly,maxpoly+1))

In [ ]:
EuropeCells = [1181, 1212, 1245, 1246, 1247, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332,
               1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1378, 1379, 1380, 1381, 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389,
               1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446,
               1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505,
               1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574,
               1575, 1576, 1577, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1677, 1678,
               1679, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1700, 1746, 1747, 1748,
               1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1824,
               1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849,
               1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859]

EuropeExtraCells = [4107,4108,3751,4808,5107,4805,4855,4755,4854,4904,4954,5004,5053,5054,5055,5102,5103,5104,5105,5152,5153,5154,5155,5156]

NoEuropeCells = [1, 10, 100, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 101, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 102, 1020,
                 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 103, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 104, 1040, 1041, 1042, 1043,
                 1044, 1045, 1046, 1047, 1048, 1049, 105, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 106, 1060, 1061, 1062, 1063, 1064, 1065, 1066,
                 1067, 1068, 1069, 107, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 108, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089,
                 109, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 11, 110, 1100, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 111, 1110,
                 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 112, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 113, 1130, 1131, 1132, 1133,
                 1134, 1135, 1136, 1137, 1138, 1139, 114, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 115, 1150, 1151, 1152, 1153, 1154, 1155, 1156,
                 1157, 1158, 1159, 116, 1160, 1161, 1162, 1163, 1164, 1165, 1166, 1167, 1168, 1169, 117, 1170, 1171, 1172, 1173, 1174, 1175, 1176, 1177, 1178, 1179,
                 118, 1180, 1181, 1182, 1183, 1184, 1185, 1186, 1187, 1188, 1189, 119, 1190, 1191, 1192, 1193, 1194, 1195, 1196, 1197, 1198, 1199, 12, 120, 1200,
                 1201, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 121, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 122, 1220, 1221, 1222, 1223,
                 1224, 1225, 1226, 1227, 1228, 1229, 123, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 124, 1240, 1241, 1242, 1243, 1244, 1245, 1246,
                 1247, 1248, 1249, 125, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 126, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269,
                 127, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 128, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 129, 1290, 1291,
                 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 13, 130, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 131, 1310, 1311, 1312, 1313,
                 1314, 1315, 1316, 1317, 1318, 1319, 132, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 133, 1330, 1331, 1332, 1333, 1334, 1335, 1336,
                 1337, 1338, 1339, 134, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 135, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359,
                 136, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 137, 1370, 1371, 1372, 1373, 1374, 1375, 1376, 1377, 1378, 1379, 138, 1380, 1381,
                 1382, 1383, 1384, 1385, 1386, 1387, 1388, 1389, 139, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1398, 1399, 14, 140, 1400, 1401, 1402, 1403,
                 1404, 1405, 1406, 1407, 1408, 1409, 141, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 142, 1420, 1421, 1422, 1423, 1424, 1425, 1426,
                 1427, 1428, 1429, 143, 1430, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 144, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449,
                 145, 1450, 1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 146, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468, 1469, 147, 1470, 1471,
                 1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 148, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 149, 1490, 1491, 1492, 1493, 1494,
                 1495, 1496, 1497, 1498, 1499, 15, 150, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 151, 1510, 1511, 1512, 1513, 1514, 1515, 1516,
                 1517, 1518, 1519, 152, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 153, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538, 1539,
                 154, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 155, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 156, 1560, 1561,
                 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 157, 1570, 1571, 1572, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 158, 1580, 1581, 1582, 1583, 1584,
                 1585, 1586, 1587, 1588, 1589, 159, 1590, 1591, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 16, 160, 1600, 1601, 1602, 1603, 1604, 1605, 1606,
                 1607, 1608, 1609, 161, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 162, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629,
                 163, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 164, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 165, 1650, 1651,
                 1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 166, 1660, 1661, 1662, 1663, 1664, 1665, 1666, 1667, 1668, 1669, 167, 1670, 1671, 1672, 1673, 1674,
                 1675, 1676, 1677, 1678, 1679, 168, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1687, 1688, 1689, 169, 1690, 1691, 1692, 1693, 1694, 1695, 1696, 1697,
                 1698, 1699, 17, 170, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707, 1708, 1709, 171, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718, 1719,
                 172, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1729, 173, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 174, 1740, 1741,
                 1742, 1743, 1744, 1745, 1746, 1747, 1748, 1749, 175, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, 1758, 1759, 176, 1760, 1761, 1762, 1763, 1764,
                 1765, 1766, 1767, 1768, 1769, 177, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 178, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787,
                 1788, 1789, 179, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 18, 180, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809,
                 181, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 182, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 183, 1830, 1831,
                 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 184, 1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 185, 1850, 1851, 1852, 1853, 1854,
                 1855, 1856, 1857, 1858, 1859, 186, 1860, 1861, 1862, 1863, 1864, 1865, 1866, 1867, 1868, 1869, 187, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877,
                 1878, 1879, 188, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889, 189, 1890, 1891, 19, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
                 2, 20, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 21, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22, 220, 221, 222, 223, 224, 225,
                 226, 227, 228, 229, 23, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 24, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 25, 250, 251, 252,
                 253, 254, 255, 256, 257, 258, 259, 26, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 27, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 28,
                 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 29, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 3, 30, 300, 301, 302, 303, 304, 305, 306,
                 307, 308, 309, 31, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 32, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 33, 330, 331, 332, 333,
                 334, 335, 336, 337, 338, 339, 34, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 35, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 36, 360,
                 361, 362, 363, 364, 365, 366, 367, 368, 369, 37, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 38, 380, 381, 382, 383, 384, 385, 386, 387, 388,
                 389, 39, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 4, 40, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 41, 410, 411, 412, 413, 414,
                 415, 416, 417, 418, 419, 42, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 43, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 44, 440, 441,
                 442, 443, 444, 445, 446, 447, 448, 449, 45, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 46, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469,
                 47, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 48, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 49, 490, 491, 492, 493, 494, 495, 496,
                 497, 498, 499, 5, 50, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 51, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 52, 520, 521, 522, 523,
                 524, 525, 526, 527, 528, 529, 53, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 54, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 55, 550,
                 551, 552, 553, 554, 555, 556, 557, 558, 559, 56, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 57, 570, 571, 572, 573, 574, 575, 576, 577, 578,
                 579, 58, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 59, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 6, 60, 600, 601, 602, 603, 604, 605,
                 606, 607, 608, 609, 61, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 62, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 63, 630, 631, 632,
                 633, 634, 635, 636, 637, 638, 639, 64, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 65, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 66,
                 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 67, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 68, 680, 681, 682, 683, 684, 685, 686, 687,
                 688, 689, 69, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 7, 70, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 71, 710, 711, 712, 713, 714,
                 715, 716, 717, 718, 719, 72, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 73, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 74, 740, 741,
                 742, 743, 744, 745, 746, 747, 748, 749, 75, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 76, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769,
                 77, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 78, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 79, 790, 791, 792, 793, 794, 795, 796,
                 797, 798, 799, 8, 80, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 81, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 82, 820, 821, 822, 823,
                 824, 825, 826, 827, 828, 829, 83, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 84, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 85, 850,
                 851, 852, 853, 854, 855, 856, 857, 858, 859, 86, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 87, 870, 871, 872, 873, 874, 875, 876, 877, 878,
                 879, 88, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 89, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 9, 90, 900, 901, 902, 903, 904, 905,
                 906, 907, 908, 909, 91, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 92, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 93, 930, 931, 932,
                 933, 934, 935, 936, 937, 938, 939, 94, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 95, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 96,
                 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 97, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 98, 980, 981, 982, 983, 984, 985, 986, 987,
                 988, 989, 99, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999]


In [ ]:
#polyl = list(range(1,1891)) # 1, 1891 // Marco (1-999) Gon (1000-1891)
#polyl = list(range(0,235)) # 0, 235
#polyl = list([1628,1629]) # 0, 235

#polyl = list(EuropeCells)
polyl = list(EuropeExtraCells)

In [ ]:
# Export results
for poly in polyl:

    # select one big square
    #onesquare = globalgrid.filterMetadata('FID','equals',poly).geometry().bounds()
    onesquare = globalgrid.filterMetadata('id','equals',poly).geometry().bounds()
    #onesquare = ee.FeatureCollection(globalgrid.toList(1,poly)).geometry().bounds()

    # export training data
    #task = ee.batch.Export.image.toCloudStorage(image = trainingdat,description = 'train_'+str(poly),
    #                                bucket = 'globalgedi',fileNamePrefix = 'train_'+str(poly),
    #                                scale = 25,fileFormat='GeoTIFF',
    #                                skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13,
    #                                region = onesquare) # WGS: crs ='EPSG:4326', LAEA: crs ='EPSG:3035'
    #task.start()

    task = ee.batch.Export.image.toDrive(image = trainingdat,description = 'trainEXTRA_'+str(poly),
                                     folder = 'GEDI_Global',
                                     scale = 25,fileFormat='GeoTIFF',
                                     skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13,
                                     region = onesquare)
    task.start()

    # export prediction data
    #task = ee.batch.Export.image.toCloudStorage(image = predictiondat,description = 'pred_'+str(poly),
    #                                 bucket = 'sargeditmp',fileNamePrefix = 'pred_'+str(poly),
    #                                 scale = 25,fileFormat='GeoTIFF',
    #                                 skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13,
    #                                 region = onesquare)
    #task.start()

In [ ]:
poly = 1388
onesquare = globalgrid.filterMetadata('FID','equals',poly).geometry().bounds()
# export training data
task = ee.batch.Export.image.toCloudStorage(image = trainingdat,description = 'train_'+str(poly),
                                            bucket = 'globalgedi',fileNamePrefix = 'train_'+str(poly),
                                            scale = 25,fileFormat='GeoTIFF',
                                            skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13,
                                            region = onesquare)
task.start()